# Ratterdam Linear Models
## Linear Mixed Effects Models, using Generalized framework if needed. Oct 2020. 

### Imports and global settings

In [1]:
import ratterdam_CoreDataStructures as Core
import ratterdam_ParseBehavior as Parse
import numpy as np
from scipy.stats import sem, binom_test, shapiro, bartlett
import utility_fx as util
import os
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import ratterdam_Defaults as Def
import ratterdam_visBasic as Vis
import ratterdam_DataFiltering as Filt
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp
from scipy.stats import kruskal

In [2]:
%qtconsole --style native
%matplotlib qt5

### Load Data

In [14]:
rat = 'R781'
expCode = 'BRD3'
datafile = f'E:\\Ratterdam\\{rat}\\{rat}{expCode}\\'
savepath = f'E:\\Ratterdam\\{rat}\\anovas\\'

alleyTracking, alleyVisits,  txtVisits, p_sess, ts_sess = Parse.getDaysBehavioralData(datafile, expCode)
clustlist = util.getClustList(datafile)
population = {}

for clust in clustlist:
    unit = Core.UnitData(clust, datafile, expCode, Def.alleyBounds, alleyVisits, txtVisits, p_sess, ts_sess)
    unit.loadData_raw()
    validalleys = []
    valid, acorr, alleys = util.checkInclusion(unit, 1)
    if valid:
        print(clust)
        unit.acorr = acorr
        unit.validAlleys = alleys
        population[clust] = unit

E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:174: RuntimeWarning: divide by zero encountered in reciprocal
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:174: RuntimeWarning: invalid value encountered in multiply
  n = (hs*np.reciprocal(ho))*30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:180: RuntimeWarning: divide by zero encountered in reciprocal
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\ratterdam_CoreDataStructures.py:180: RuntimeWarning: invalid value encountered in multiply
  n = (ls* np.reciprocal(lo)) * 30
E:\UserData\Documents\GitHub\ratterdam\utility_fx.py:329: RuntimeWarning: invalid value encountered in true_divide
  Z=VV/WW


TT11\cl-maze1.1
TT11\cl-maze1.2
TT12\cl-maze1.1
TT12\cl-maze1.2
TT3\cl-maze1.3
TT3\cl-maze1.4
TT3\cl-maze1.5
TT3\cl-maze1.6
TT3\cl-maze1.7
TT3\cl-maze1.8
TT5\cl-maze1.1
TT5\cl-maze1.3
TT6\cl-maze1.1
TT6\cl-maze1.2
TT6\cl-maze1.3
TT6\cl-maze1.4
TT6\cl-maze1.6
TT6\cl-maze1.7
TT6\cl-maze1.8
TT6\cl-maze1.9
TT9\cl-maze1.1
TT9\cl-maze1.2
TT9\cl-maze1.5
TT9\cl-maze1.6
TT9\cl-maze1.7


### Create "Long Form" of Data. Single Day
#### Nested Structure: Cell > trial > texture > reward > spatial bin. 

In [ ]:
cells, trials, textures, rewards, spatialBins = [], [], [], [], []

for cellID, (unitname, unit) in enumerate(population.items()):
    for alley in unit.validAlleys:
        for i,visit in enumerate(unit.alleys[alley]):
            cells.append(cellID)
            trials.append(i)
            